In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Data Acquisition
- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

- Read the case, department, and source data into their own spark dataframes.

In [2]:
# header = True (calls in headers from CSV)
# inferSchema = True (infers datatype of columns in csv)
case = spark.read.csv("case.csv", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", header=True, inferSchema=True)
source = spark.read.csv("source.csv", header=True, inferSchema=True)

Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [5]:
#convert to .csv files
source.write.csv("sources_csv", mode="overwrite")
dept.write.csv("depts_csv", mode="overwrite")
case.write.csv("cases_csv", mode="overwrite")
#convert to .json files
source.write.json("sources_json", mode="overwrite")
dept.write.json("depts_json", mode="overwrite")
case.write.json("cases_json", mode="overwrite")

- Inspect your folder structure. What do you notice?

Inside the folder the csv and json are stored as folders 

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [6]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [7]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [8]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [15]:
# rename to match other columns
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

# change case_closed and case_late to boolean values
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

# change city council district to string not int (categorical)
case = case.withColumn("council_district", col("council_district").cast("string"))
# add 00 before number
case = case.withColumn("council_district", col("council_district").cast("int"))

case = case.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

# Now we will handle the 3 columns that have dates in them. We'll use spark's to_timestamp function for this.
fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

# standardize address column, trim, removes whitespace, lowercase
case = case.withColumn("request_address", trim(lower(case.request_address)))

# convert number of days late to weeks
case = case.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

# new features (zipcode)
case = case.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))


case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [16]:
#Join data frames

df = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [17]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1229                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [19]:
# How old is the latest currently open issue
(
df.select('SLA_days','case_opened_date','case_due_date','case_lifetime','case_age','num_days_late')
  .where(df.case_closed == False)
  .sort(df.case_opened_date.desc())
  .show(1)
)

+--------+-------------------+-------------------+-------------+--------+-------------+
|SLA_days|   case_opened_date|      case_due_date|case_lifetime|case_age|num_days_late|
+--------+-------------------+-------------------+-------------+--------+-------------+
|    14.0|2018-08-08 10:38:00|2018-08-22 10:38:00|         1010|    1010| -13.44305556|
+--------+-------------------+-------------------+-------------+--------+-------------+
only showing top 1 row



- How many Stray Animal cases are there?

In [20]:
(
df.where(df.department == 'Animal Care Services')
  .agg(count('case_id'))
  .show()
)

+--------------+
|count(case_id)|
+--------------+
|        116915|
+--------------+



- How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [21]:
(
    df.filter(col("service_request_type") != lit("Officer Standby"))
    .count()
)

838691

- Convert the council_district column to a string column.

In [26]:
case = case.withColumn("council_district", col("council_district").cast("string"))

- Extract the year from the case_closed_date column.

In [28]:
df = df.withColumn("year_closed", year("case_closed_date"))
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1229                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 year                 | 2018      

Convert num_days_late from days to hours in new columns num_hours_late.

In [30]:
df = df.withColumn("num_hours_late", (col("num_days_late") * 24))
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1229                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 year                 | 2018      

Join the case data with the source and department data.

In [31]:
#Join data frames

df = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

Are there any cases that do not have a request source?

In [32]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



What are the top 10 service request types in terms of number of requests?

In [33]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



What are the top 10 service request types in terms of average days late?

In [39]:
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('#_days_late'), count('*').alias('number_of_cases'))
    .sort(desc('#_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+---------------+
|service_request_type                  |#_days_late       |number_of_cases|
+--------------------------------------+------------------+---------------+
|Zoning: Recycle Yard                  |210.89201994318182|132            |
|Zoning: Junk Yards                    |200.20517608494276|262            |
|Structure/Housing Maintenance         |190.20707698509804|51             |
|Donation Container Enforcement        |171.09115313942618|122            |
|Storage of Used Mattress              |163.96812829714287|7              |
|Labeling for Used Mattress            |162.43032902285717|7              |
|Record Keeping of Used Mattresses     |153.99724039428568|7              |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12             |
|Traffic Signal Graffiti               |137.64583330000002|2              |
|License Requied Used Mattress Sales   |128.79828704142858|7              |
+-----------

Does number of days late depend on department?

In [40]:
(
    df.filter('case_late')
    .groupby('department')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+------------------------+---------+------------+
|department              |days_late|n_cases_late|
+------------------------+---------+------------+
|Metro Health            |6.5      |829         |
|Solid Waste             |7.2      |32945       |
|Trans & Cap Improvements|10.6     |5411        |
|Parks & Recreation      |22.3     |3797        |
|Animal Care Services    |23.5     |23276       |
|DSD/Code Enforcement    |49.4     |26235       |
|Customer Service        |87.7     |2010        |
+------------------------+---------+------------+



How do number of days late depend on department and request type?

In [41]:
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("department", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

+--------------------+--------------------------------------------+---------+-------+
|department          |service_request_type                        |days_late|n_cases|
+--------------------+--------------------------------------------+---------+-------+
|DSD/Code Enforcement|Zoning: Recycle Yard                        |273.6    |75     |
|DSD/Code Enforcement|Zoning: Junk Yards                          |251.9    |146    |
|DSD/Code Enforcement|Donation Container Enforcement              |201.7    |82     |
|DSD/Code Enforcement|Structure/Housing Maintenance               |182.4    |30     |
|DSD/Code Enforcement|Graffiti: Private Property (Corridors)      |175.1    |3      |
|DSD/Code Enforcement|Storage of Used Mattress                    |164.0    |7      |
|DSD/Code Enforcement|Labeling for Used Mattress                  |162.4    |7      |
|DSD/Code Enforcement|Record Keeping of Used Mattresses           |154.0    |7      |
|DSD/Code Enforcement|Signage Requied for Sale of Used